In [27]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
import time
import os

from dgl.geometry import farthest_point_sampler
import open3d as o3d

from utils import fps, depth2fgpcd, pcd2pix, fps_np, downsample_pcd

In [28]:
env = 'test_carrots'
cam_view = 'cam_view_4'
dir_path = os.path.join(env, cam_view)
dir_path

'test_carrots/cam_view_4'

In [29]:
raw_obs = np.load(os.path.join(dir_path, 'obs.npy'))
raw_obs.shape

(720, 720, 5)

In [30]:
camera_intrinsic_params = np.load(os.path.join(dir_path, 'camera_intrinsic_params.npy'))
camera_intrinsic_params

array([869.11683083, 869.11683083, 360.        , 360.        ])

In [31]:
global_scale = 24
obs = raw_obs
depth = obs[..., -1] / global_scale
# fgpcd = depth2fgpcd(depth, depth<0.599/0.8, camera_intrinsic_params)
# fgpcd = downsample_pcd(fgpcd, 0.01)


In [32]:
fgpcd = depth2fgpcd(depth, depth<0.599/0.8, camera_intrinsic_params)

In [33]:
print(depth.shape)
print(fgpcd.shape)

(720, 720)
(71098, 3)


In [34]:
depth<0.599/0.8

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [35]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(fgpcd)
o3d.visualization.draw_geometries([pcd])

In [36]:
vis = o3d.visualization.Visualizer()
vis.create_window()
vis.get_render_option().point_color_option = o3d.visualization.PointColorOption.Color
vis.get_render_option().point_size = 3.0
vis.add_geometry(pcd)
vis.capture_screen_image(cam_view + ".jpg", do_render=True)
vis.destroy_window()

In [37]:
# save to image
o3d.io.write_point_cloud(os.path.join(dir_path, 'fgpcd.pcd'), pcd)

True

In [38]:
# point cloud video
# vis = o3d.visualization.Visualizer()
# vis.create_window()

# # geometry is the point cloud used in your animaiton
# geometry = o3d.geometry.PointCloud()
# vis.add_geometry(geometry)

# global_scale = 24
# particle_num = 50 # obtained from resolution regressor 
# save_images = True

# for i in range(len(raw_obs)):
#     obs = raw_obs[i]
#     depth = obs[..., -1] / global_scale
    
#     fgpcd = depth2fgpcd(depth, depth<0.599/0.8, camera_params)
#     fgpcd = downsample_pcd(fgpcd, 0.01)
#     sampled_ptcl, particle_r = fps(fgpcd, particle_num) 
#     fgpcd = sampled_ptcl
    
#     geometry.points = o3d.utility.Vector3dVector(fgpcd)
#     vis.add_geometry(geometry)
#     vis.poll_events()
#     vis.update_renderer()
    
#     if save_images:
#         vis.capture_screen_image("imgs/{:04d}.jpg".format(i))
    
#     time.sleep(0.3)

# vis.destroy_window()

In [39]:
# # imgs to video
# import glob 
# import os

# img_array = []
# for filename in sorted(glob.glob(f'./imgs/*.jpg'), key=os.path.getmtime):
#     img = cv2.imread(filename)
#     height, width, layers = img.shape
#     size = (width,height)
#     img_array.append(img)

# out = cv2.VideoWriter(f'ptcl_sampled.mp4',cv2.VideoWriter_fourcc(*'mp4v'), 5, size)

# for i in range(len(img_array)):
#     out.write(img_array[i])
# out.release()
